<a href="https://colab.research.google.com/github/ayyucedemirbas/Model_Serving/blob/main/TorchScript_for_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import torchvision.models as models

In [2]:
r18 = models.resnet18(pretrained=True)       # We now have an instance of the pretrained model
r18_scripted = torch.jit.script(r18)         # *** This is the TorchScript export
dummy_input = torch.rand(1, 3, 224, 224)     # We should run a quick test

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [3]:
unscripted_output = r18(dummy_input)         # Get the unscripted model's prediction...
scripted_output = r18_scripted(dummy_input)  # ...and do the same for the scripted version

unscripted_top5 = F.softmax(unscripted_output, dim=1).topk(5).indices
scripted_top5 = F.softmax(scripted_output, dim=1).topk(5).indices

print('Python model top 5 results:\n  {}'.format(unscripted_top5))
print('TorchScript model top 5 results:\n  {}'.format(scripted_top5))

Python model top 5 results:
  tensor([[463, 600, 731, 412, 618]])
TorchScript model top 5 results:
  tensor([[463, 600, 731, 412, 618]])


In [4]:
r18_scripted.save('r18_scripted.pt')

In [5]:
%%writefile ts-infer.cpp

#include <torch/script.h>
#include <torch/nn/functional/activation.h>


int main(int argc, const char* argv[]) {
    if (argc != 2) {
        std::cerr << "usage: ts-infer <path-to-exported-model>\n";
        return -1;
    }

    std::cout << "Loading model...\n";

    // deserialize ScriptModule
    torch::jit::script::Module module;
    try {
        module = torch::jit::load(argv[1]);
    } catch (const c10::Error& e) {
        std::cerr << "Error loading model\n";
        std::cerr << e.msg_without_backtrace();
        return -1;
    }

    std::cout << "Model loaded successfully\n";

    torch::NoGradGuard no_grad; // ensures that autograd is off
    module.eval(); // turn off dropout and other training-time layers/functions

    // create an input "image"
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(torch::rand({1, 3, 224, 224}));

    // execute model and package output as tensor
    at::Tensor output = module.forward(inputs).toTensor();

    namespace F = torch::nn::functional;
    at::Tensor output_sm = F::softmax(output, F::SoftmaxFuncOptions(1));
    std::tuple<at::Tensor, at::Tensor> top5_tensor = output_sm.topk(5);
    at::Tensor top5 = std::get<1>(top5_tensor);

    std::cout << top5[0] << "\n";

    std::cout << "\nDONE\n";
    return 0;
}


Writing ts-infer.cpp


In [6]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(custom_ops)

find_package(Torch REQUIRED)

add_executable(ts-infer ts-infer.cpp)
target_link_libraries(ts-infer "${TORCH_LIBRARIES}")
set_property(TARGET ts-infer PROPERTY CXX_STANDARD 11)

Writing CMakeLists.txt


In [7]:
!wget https://download.pytorch.org/libtorch/nightly/cpu/libtorch-shared-with-deps-latest.zip

--2023-03-23 09:55:36--  https://download.pytorch.org/libtorch/nightly/cpu/libtorch-shared-with-deps-latest.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.33.88.63, 13.33.88.59, 13.33.88.36, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.33.88.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192468768 (184M) [application/zip]
Saving to: ‘libtorch-shared-with-deps-latest.zip’

libtorch-shared-wit 100%[===================>] 183.55M   331MB/s    in 0.6s    

2023-03-23 09:55:37 (331 MB/s) - ‘libtorch-shared-with-deps-latest.zip’ saved [192468768/192468768]



In [8]:
!unzip libtorch-shared-with-deps-latest.zip

Streaming output truncated to the last 5000 lines.
  inflating: libtorch/include/ATen/ops/fft_irfftn_native.h  
  inflating: libtorch/include/ATen/ops/fft_irfftn_ops.h  
  inflating: libtorch/include/ATen/ops/fft_rfft.h  
  inflating: libtorch/include/ATen/ops/fft_rfft2.h  
  inflating: libtorch/include/ATen/ops/fft_rfft2_compositeimplicitautograd_dispatch.h  
  inflating: libtorch/include/ATen/ops/fft_rfft2_native.h  
  inflating: libtorch/include/ATen/ops/fft_rfft2_ops.h  
  inflating: libtorch/include/ATen/ops/fft_rfft_compositeimplicitautograd_dispatch.h  
  inflating: libtorch/include/ATen/ops/fft_rfft_native.h  
  inflating: libtorch/include/ATen/ops/fft_rfft_ops.h  
  inflating: libtorch/include/ATen/ops/fft_rfftfreq.h  
  inflating: libtorch/include/ATen/ops/fft_rfftfreq_compositeexplicitautograd_dispatch.h  
  inflating: libtorch/include/ATen/ops/fft_rfftfreq_native.h  
  inflating: libtorch/include/ATen/ops/fft_rfftfreq_ops.h  
  inflating: libtorch/include/ATen/ops/fft_rfftn

In [10]:
!cmake -DCMAKE_PREFIX_PATH="./libtorch"

CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


-- Found Torch: /content/libtorch/lib/libtorch.so  
-- Configuring done
-- Generating done
-- Build files have been written to: /content


In [11]:
!make

Streaming output truncated to the last 5000 lines.
  205 |         c10::optional<Device>(c10::in_place, std::forward<Args>(args)...));
      |              ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from /content/libtorch/include/c10/core/SymIntArrayRef.h:6,
                 from /content/libtorch/include/c10/core/impl/PyInterpreter.h:6,
                 from /content/libtorch/include/c10/core/impl/PyObjectSlot.h:4,
                 from /content/libtorch/include/c10/core/StorageImpl.h:6,
                 from /content/libtorch/include/c10/core/Storage.h:3,
                 from /content/libtorch/include/c10/core/TensorImpl.h:8,
                 from /content/libtorch/include/c10/core/GeneratorImpl.h:8,
                 from /content/libtorch/include/ATen/core/Generator.h:22,
                 from /content/libtorch/include/ATen/CPUGeneratorImpl.h:3,
                 from /content/libtorch/include/ATen/Context.h:3,
                 from /content/libtor